
# Boston Housing Analysis — Management Insights

**Project Scenario:** You are a Data Scientist with a housing agency in Boston, MA. Using the Boston Housing dataset (derived from the U.S. Census Service), provide insights to management.

**Tasks covered by this notebook**  
- Task 1: Familiarize with the dataset  
- Task 2: Generate basic statistics and visualizations  
- Task 3: Run statistical tests to answer management questions  
- Task 4: Share this Jupyter Notebook

**Alpha level:** $\alpha = 0.05$


## Setup & Data Loading

In [ ]:

# Install deps if needed (uncomment if running in a very minimal environment)
# %pip install pandas scipy statsmodels matplotlib

import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Display options
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# ---- Data loading with 3 strategies ----
# 1) Default: Load from IBM Skills Network (works on regular internet-connected environments).
# 2) If that fails, try to read a local file dropped next to the notebook (boston_housing.csv).
# 3) If that fails too, raise an informative error.

def load_boston():
    url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv"
    try:
        df = pd.read_csv(url)
        print("Loaded dataset from URL.")
        return df
    except Exception as e:
        print("URL load failed:", e)
        try:
            df = pd.read_csv("boston_housing.csv")
            print("Loaded local boston_housing.csv (placed next to this notebook).")
            return df
        except Exception as e2:
            raise RuntimeError(
                "Unable to load the dataset from the URL or a local 'boston_housing.csv'. "
                "Please place the CSV next to this notebook and re-run."
            ) from e2

boston_df = load_boston()

# Some distributions include an 'Unnamed: 0' index column; drop if present
if "Unnamed: 0" in boston_df.columns:
    boston_df = boston_df.drop(columns=["Unnamed: 0"])

boston_df.head()



### Data Dictionary

- **CRIM** — per capita crime rate by town  
- **ZN** — proportion of residential land zoned for lots over 25,000 sq.ft.  
- **INDUS** — proportion of non-retail business acres per town  
- **CHAS** — Charles River dummy variable (1 if tract bounds river; 0 otherwise)  
- **NOX** — nitric oxides concentration (parts per 10 million)  
- **RM** — average number of rooms per dwelling  
- **AGE** — proportion of owner-occupied units built prior to 1940  
- **DIS** — weighted distances to five Boston employment centres  
- **RAD** — index of accessibility to radial highways  
- **TAX** — full-value property-tax rate per \$10,000  
- **PTRATIO** — pupil–teacher ratio by town  
- **LSTAT** — % lower status of the population  
- **MEDV** — median value of owner-occupied homes in \$1000's


## Task 1–2: Familiarize & Generate Basic Statistics

In [ ]:

print("Shape:", boston_df.shape)
display(boston_df.describe(include="all"))
print("\nMissing values per column:")
print(boston_df.isna().sum())


### Visualizations

In [ ]:

# IMPORTANT: Use matplotlib only, one chart per cell, and avoid setting specific colors.

# 1) Boxplot: MEDV (Median value of owner-occupied homes)
plt.figure()
plt.boxplot(boston_df["MEDV"].dropna())
plt.title("Median Value of Owner-Occupied Homes (MEDV) — Boxplot")
plt.ylabel("MEDV ($1000s)")
plt.xlabel("All observations")
plt.show()


In [ ]:

# 2) Bar plot for CHAS (0/1 counts)
chas_counts = boston_df["CHAS"].value_counts().sort_index()
plt.figure()
plt.bar(chas_counts.index.astype(str), chas_counts.values)
plt.title("Bounded by the Charles River (CHAS) — Counts")
plt.xlabel("CHAS (0=No, 1=Yes)")
plt.ylabel("Count of Tracts")
plt.show()


In [ ]:

# 3) Boxplot: MEDV vs AGE groups
# Discretize AGE into 3 bins: <=35, (35, 70], >70
age_bins = pd.cut(boston_df["AGE"],
                  bins=[-np.inf, 35, 70, np.inf],
                  labels=["<=35", "35–70", ">70"])

groups = [boston_df.loc[age_bins == lab, "MEDV"].dropna() for lab in ["<=35", "35–70", ">70"]]

plt.figure()
plt.boxplot(groups, labels=["<=35", "35–70", ">70"])
plt.title("MEDV by AGE Group — Boxplots")
plt.xlabel("AGE Group (% owner-occupied units built prior to 1940)")
plt.ylabel("MEDV ($1000s)")
plt.show()


In [ ]:

# 4) Scatter: NOX vs INDUS
plt.figure()
plt.scatter(boston_df["INDUS"], boston_df["NOX"])
plt.title("NOX vs INDUS — Scatter Plot")
plt.xlabel("INDUS (% non-retail business acres per town)")
plt.ylabel("NOX (parts per 10 million)")
plt.show()

# Simple Pearson correlation to accompany the visual
r, p = stats.pearsonr(boston_df["INDUS"], boston_df["NOX"])
print(f"Pearson r between NOX and INDUS: {r:.3f}, p-value: {p:.3g}")
print("Interpretation: r close to 0 => weak linear relationship; |r| close to 1 => strong linear relationship.")


In [ ]:

# 5) Histogram: PTRATIO
plt.figure()
plt.hist(boston_df["PTRATIO"].dropna(), bins=15)
plt.title("Pupil–Teacher Ratio (PTRATIO) — Histogram")
plt.xlabel("PTRATIO")
plt.ylabel("Frequency")
plt.show()


## Task 3: Statistical Tests


### Q1) Do houses bounded by the Charles River differ in median value? (Independent samples t-test)

- **H₀:** Mean MEDV is the same for CHAS=1 and CHAS=0.  
- **H₁:** Mean MEDV differs between CHAS=1 and CHAS=0.  
- **α = 0.05**.


In [ ]:

# Split groups
medv_chas1 = boston_df.loc[boston_df["CHAS"] == 1, "MEDV"].dropna()
medv_chas0 = boston_df.loc[boston_df["CHAS"] == 0, "MEDV"].dropna()

# Welch's t-test (unequal variances)
t_stat, p_val = stats.ttest_ind(medv_chas1, medv_chas0, equal_var=False)
print(f"T-stat: {t_stat:.3f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Conclusion: Reject H0. There is a statistically significant difference in MEDV between river-bounded and non-river tracts.")
else:
    print("Conclusion: Fail to reject H0. No statistically significant difference detected at α=0.05.")



### Q2) Is there a difference in MEDV across AGE groups? (One-way ANOVA)

AGE groups:  
- ≤ 35% (<=35)  
- 35–70% (35–70)  
- > 70% (>70)

- **H₀:** All group means of MEDV are equal.  
- **H₁:** At least one group mean differs.  
- **α = 0.05**.


In [ ]:

age_bins = pd.cut(boston_df["AGE"], bins=[-np.inf, 35, 70, np.inf], labels=["<=35", "35–70", ">70"])
g1 = boston_df.loc[age_bins=="<=35", "MEDV"].dropna()
g2 = boston_df.loc[age_bins=="35–70", "MEDV"].dropna()
g3 = boston_df.loc[age_bins==">70", "MEDV"].dropna()

F_stat, p_val = stats.f_oneway(g1, g2, g3)
print(f"F-stat: {F_stat:.3f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Conclusion: Reject H0. There is a statistically significant difference in MEDV across AGE groups.")
else:
    print("Conclusion: Fail to reject H0. No statistically significant difference detected at α=0.05.")



### Q3) Is there no relationship between NOX and INDUS? (Pearson correlation)

- **H₀:** ρ = 0 (no linear relationship).  
- **H₁:** ρ ≠ 0 (there is a linear relationship).  
- **α = 0.05**.


In [ ]:

r, p_val = stats.pearsonr(boston_df["NOX"], boston_df["INDUS"])
print(f"Pearson r: {r:.3f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Conclusion: Reject H0. There is evidence of a linear relationship between NOX and INDUS.")
else:
    print("Conclusion: Fail to reject H0. No evidence of a linear relationship at α=0.05.")



### Q4) Impact of an additional unit of DIS on MEDV (Simple Linear Regression)

Model: **MEDV ~ DIS**  
- **H₀:** Slope( DIS ) = 0 (no impact).  
- **H₁:** Slope( DIS ) ≠ 0 (non-zero impact).  
- **α = 0.05**.

Interpret the slope: the expected change in MEDV (in \$1000s) for a one–unit increase in DIS.


In [ ]:

model = smf.ols("MEDV ~ DIS", data=boston_df).fit()
display(model.summary())

slope = model.params["DIS"]
p_val = model.pvalues["DIS"]
print(f"Slope for DIS: {slope:.3f} ($1000s per unit of DIS), p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Conclusion: Reject H0. DIS has a statistically significant impact on MEDV.")
else:
    print("Conclusion: Fail to reject H0. No significant impact detected at α=0.05.")
